<a href="https://colab.research.google.com/github/pankajhpatil/AlternusVera_Education/blob/main/AlternusVera_Education.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import Counter
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
import numpy as np
import re
import statistics
import urllib.request,sys,time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
import os

Label_map={0: 'barely-true',
 1: 'false',
 2: 'full-flop',
 3: 'half-flip',
 4: 'half-true',
 5: 'mostly-true',
 6: 'pants-fire',
 7: 'true'}

speaker_map={0: 'Barack Obama',
 1: 'Chain email',
 2: 'Chris Christie',
 3: 'Donald Trump',
 4: 'Hillary Clinton',
 5: 'John McCain',
 6: 'Marco Rubio',
 7: 'Mitt Romney',
 8: 'Rick Perry',
 9: 'Scott Walker'}

def getTokerns(txt):
  tokenized = sent_tokenize(txt)
  for i in tokenized:
      wordsList = nltk.word_tokenize(i)
      wordsList = [w for w in wordsList if not w in stop_words]
      tagged = nltk.pos_tag(wordsList)
      counts = Counter(tag for (word, tag) in tagged)
      total = sum(counts.values())
      a = dict((word, float(count) / total) for (word, count) in
              counts.items())
      return a;

def predictLable(text):
  data = {'Statement':  [text]
        }
  df_test = pd.DataFrame (data, columns = ['Statement'])
  
  postags = ['CC','CD','DT','EX','FW','IN','JJ','JJR','JJS','LS','MD','NN','NNS','NNP','NNPS','PDT','POS','PRP','PRP$','RB','RBR','RBS','RP','SYM','TO','UH','VB','VBD','VBG','VBN','VBP','VBZ','WDT','WP','WP$','WRB']

  for i,txt in enumerate(postags):
    df_test[txt]=0.00;

  for i,txt in enumerate(df_test['Statement']):
    a = getTokerns(txt)
    for key in a:
        if key in postags:
          df_test[key][i]=a[key]
  
  spearker_df = df_test.filter(items=['CC', 'CD',
       'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS',
       'NNP', 'NNPS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP',
       'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP',
       'WP$', 'WRB'])
  

  cwd = os.getcwd()
  print(cwd)

  with open('./knn_speaker_Model.pkl', 'rb') as file:  
    knn_speaker_Model = pickle.load(file)
  
  # print(spearker_df)
  sp = knn_speaker_Model.predict(spearker_df)

  res = sp[0,0];
  if res - int(res) > 0.49:
    res=int(res)+1
  else:
    res=int(res)

  src=speaker_map.get(res)

  print(src)  
  # print(df_test)
  for key in speaker_map:
    if speaker_map[key] == src:
      df_test['Source_cat']=key
  
  with open('./knn_truth_Model.pkl', 'rb') as file:  
    knn_truth_Model = pickle.load(file)

  source_df = df_test.filter(items=['Source_cat', 'CC', 'CD',
       'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS',
       'NNP', 'NNPS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP',
       'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP',
       'WP$', 'WRB'])
  pred=knn_truth_Model.predict(source_df)

  res_label = pred[0,0];
  if res_label - int(res_label) > 0.49:
    res_label=int(res_label)+1
  else:
    res_label=int(res_label)

  # print(pred[0,0])
  return Label_map.get(res_label)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
# text='Ann and I extend our congratulations to President-elect Joe Biden and Vice President-elect Kamala Harris. We know both of them as people of good will and admirable character. We pray that God may bless them in the days and years ahead.'

# text='America, I’m honored that you have chosen me to lead our great country.The work ahead of us will be hard, but I promise you this: I will be a President for all Americans — whether you voted for me or not.I will keep the faith that you have placed in me.'
# result = predictLable(text)
# print(result)

/content
Donald Trump
pants-fire


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
